<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [102]:
import geocoder
from scipy.stats import norm
import requests, zipfile, io, os
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import sys, typing
from bs4 import BeautifulSoup
import requests
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import matplotlib.dates as mdates
from typing import List, Tuple
r = requests.get('https://www.ssa.gov/oact/babynames/names.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("names")

In [188]:
# Import dataset of users with age, profession, height, location, and gender 
actual = pd.read_csv("actual_data.csv").drop(columns=['Link', 'Id'])
actual["Location"] = actual["Location"].str.split(",|/").str[0].str.strip()
actual["Location"] = actual["Location"].str.split("and").str[0].str.strip()
actual["Location"] = actual["Location"].replace({"Roch": "Rochester"})
actual["Location"] = actual["Location"].replace({"NYC": "New York City"})
actual["Location"] = actual["Location"].replace({"West Michigan" : "Michigan"})
actual["Gender"] = actual["Gender"].replace({"She/her": "F"})
actual["Gender"] = actual["Gender"].replace({"She.her": "F"})
actual["Gender"] = actual["Gender"].replace({"They/she": "F"})
actual["Gender"] = actual["Gender"].replace({"She/they": "F"})
actual["Gender"] = actual["Gender"].replace({"He/him": "M"})
actual["Gender"] = actual["Gender"].replace({"He/Him": "M"})
actual.loc[actual["Name "] == "Bobby", "Gender"] = "M"
actual.loc[actual["Name "] == "Sophia", "Gender"] = "F"

actual["Name"] = actual["Name "]
actual = actual.drop(columns=["Name "])
states = pd.read_csv("csvData.csv")[["State", "Pop"]]
print(actual)

    Age Height Gender        Location  \
0    22    6'0      F           Texas   
1    24    5'2      F       Las Vegas   
2    13    5'9      F           Texas   
3    14    5'1      F      New Jersey   
4    24    5'1      F        Illinois   
5    17    5'5      F     Kansas City   
6    21    5'3      F      Pittsburgh   
7    29    5'2      F      Washington   
8    20    5'0      F           Texas   
9    23    5'6      F       Kalamazoo   
10   22    5'6      F         Florida   
11   16    5'6      F            Utah   
12   27    6'3      M            Reno   
13   18    5'6      F      California   
14   24  5'5.9      F       Rochester   
15   36    5'9      M   Moreno Valley   
16   18    5'4      F        Kentucky   
17   29    5'4      F        Illinois   
18   16    6'0      M        New York   
19   22    5'0      F          Kansas   
20   20   5'10      M            Ohio   
21   30    5'3      F          Alaska   
22   28    5'8      F      New Jersey   
23   19    5'9  

In [189]:
import qwikidata
import qwikidata.sparql

def get_city_wikidata(city, country):
    query = """
    SELECT ?city ?cityLabel ?country ?countryLabel ?population
    WHERE
    {
      ?city rdfs:label '%s'@en.
      ?city wdt:P1082 ?population.
      ?city wdt:P17 ?country.
      ?city rdfs:label ?cityLabel.
      ?country rdfs:label ?countryLabel.
      FILTER(LANG(?cityLabel) = "en").
      FILTER(LANG(?countryLabel) = "en").
      FILTER(CONTAINS(?countryLabel, "%s")).
    }
    """ % (city, country)

    res = qwikidata.sparql.return_sparql_query_results(query)
    out = res['results']['bindings'][0]
    return out


In [190]:
# Location


import math
import geonamescache
gc = geonamescache.GeonamesCache()
num_unique = []
for i, row in enumerate(actual.iterrows()):
    g = gc.search_cities(row[1]["Location"])
    if not g:
        try:
            g = get_city_wikidata(row[1]["Location"], 'America')
        except:
            try: 
                g = states[states["State"] == row[1]["Location"]]["Pop"].iloc[0]
            except:
                g = float("NaN")      
    else:
        g = int(g[0]["population"])
    if isinstance(g, dict):
        g = int(g['population']['value'])
    num_unique.append(g)

actual["unique"] = num_unique
print(actual)

    Age Height Gender        Location  \
0    22    6'0      F           Texas   
1    24    5'2      F       Las Vegas   
2    13    5'9      F           Texas   
3    14    5'1      F      New Jersey   
4    24    5'1      F        Illinois   
5    17    5'5      F     Kansas City   
6    21    5'3      F      Pittsburgh   
7    29    5'2      F      Washington   
8    20    5'0      F           Texas   
9    23    5'6      F       Kalamazoo   
10   22    5'6      F         Florida   
11   16    5'6      F            Utah   
12   27    6'3      M            Reno   
13   18    5'6      F      California   
14   24  5'5.9      F       Rochester   
15   36    5'9      M   Moreno Valley   
16   18    5'4      F        Kentucky   
17   29    5'4      F        Illinois   
18   16    6'0      M        New York   
19   22    5'0      F          Kansas   
20   20   5'10      M            Ohio   
21   30    5'3      F          Alaska   
22   28    5'8      F      New Jersey   
23   19    5'9  

In [193]:
# First Name, Age
# Assumes that videos are posted this year for age
# Assumes location is in the US and area follows normal distribution for ages in the US 
# Use problem set one to set the probability that someone with their name and gender would be there age 
US_pop = 329500000

def create_name_popularity_df(name_list: List[str], sex: str) -> pd.DataFrame:
    # Formats and extracts for each year the amount of people born with a particular name.
    # INPUT
    #   name_list : list of names. They must be of the same gender
    #   sex : 'F' for Female, 'M' for Male. 
    # OUTPUT
    #   names_df: pandas DataFrame indexed by year with names for columns
    names_df = pd.DataFrame()
    for year in range(1900, 2021):
        year_table = pd.read_csv("names/yob%s.txt" % str(year),sep=',',header=None,names=['name','sex','n'])
        year_table = year_table[year_table['name'].isin(name_list)]
        year_table = year_table[year_table['sex']==sex]
        if year_table.shape[0]==0:
            year_table['name']=name_list
        year_table['year'] = year
        names_df = names_df.append(year_table)
    names_df = names_df.pivot(index='year', columns='name', values='n')
    names_df = names_df.fillna(0)
    return names_df

for row in actual.iterrows():
    df = create_name_popularity_df([row[1]["Name"]], row[1]["Gender"])
    year_born = 2020-int(row[1]["Age"])
    number_of_person_with_name_age = df.loc[df.index == year_born][row[1]["Name"]].iloc[0]
    print(number_of_person_with_name_age)
    
    # Assumes age and name distribution are consistent everywhere 
    #actual.loc[row[0],'unique'] = number_of_person_with_name_age*actual.loc[row[0],'unique']/US_pop
    actual.loc[row[0],'unique_age_name']  = number_of_person_with_name_age

print(actual)


1058.0
0.0
331.0
22.0
883
0.0
0.0
20309
1753.0
0.0
186
13.0
28732.0
8665
556
30517
12325.0
25229
22208.0
0.0
730.0
8.0
2745
4274.0
6144
20.0
0.0
3908
32.0
15426
23.0
57.0
0.0
13473
11255
0.0
56.0
638.0
1054
0.0
46.0
18823
114
8176
755.0
1516.0
558.0
4479
0.0
0.0
    Age Height Gender        Location  \
0    22    6'0      F           Texas   
1    24    5'2      F       Las Vegas   
2    13    5'9      F           Texas   
3    14    5'1      F      New Jersey   
4    24    5'1      F        Illinois   
5    17    5'5      F     Kansas City   
6    21    5'3      F      Pittsburgh   
7    29    5'2      F      Washington   
8    20    5'0      F           Texas   
9    23    5'6      F       Kalamazoo   
10   22    5'6      F         Florida   
11   16    5'6      F            Utah   
12   27    6'3      M            Reno   
13   18    5'6      F      California   
14   24  5'5.9      F       Rochester   
15   36    5'9      M   Moreno Valley   
16   18    5'4      F        Kentucky   

In [181]:
# Profession
# Height

In [8]:
import statistics
df = [0]*49 + [1]
print(df)
st_dev = statistics.pstdev(df)
print(st_dev)

1.96*st_dev/(50**.5)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
0.13999999999999999


0.038806020151517726